<a href="https://colab.research.google.com/github/choi-yongsuk/deep-learning-nlp/blob/master/imdb_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras import utils

# IMDB 데이터다운로드

dtat_set = tf.keras.utils.get_file(
      fname="imdb.tar.gz", #downloaded file name
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
      extract=True)

def dictory_data(dirctory):
    data = {}
    data["review"] = []
    for file_path in os.listdir(directory):
      with open(os.path.join(directory, file_path), "r",encoding='utf-8') as file:
        data["review"].append(file.read())
    return pd.DataFrame.from_dict(data)

def data(directory):
    pos_df = directory_data(os.path.join(directory, "pos"))
    neg_df = directory_Data(os.path.join(directory, "neg"))
    pos_df["sentiment"] = 1
    neg_df["sentiment"] = 0
    return pd.concat)[pos_df, neg_df]

#학습    
train_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "train"))

#테스를
test_df = data(os.path.join(os.path.dirname(data_set), "aclImdb", "test"))

train_df.head()
reviews = list(train_df['review'])

# 문자열 문장 리스트를 토큰화
to
